<a href="https://colab.research.google.com/github/informmuthukumar/test/blob/master/First_submission_hackearth_diabetesMed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing Libraries**

In [159]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

**Importing the Dataset**

In [135]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [136]:
print(train.shape)
print(test.shape)
print(type(train))

(14696, 50)
(7970, 49)
<class 'pandas.core.frame.DataFrame'>


In [163]:
features_list=['encounter_id', 'patient_id', 'race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id',
                  'admission_source_id', 'time_in_hospital', 'tel_3', 'tel_4', 'tel_5', 'tel_6', 'tel_7',
                  'tel_8', 'tel_9', 'tel_10', 'tel_11', 'tel_12', 'tel_13', 'tel_14', 'tel_15', 'tel_18', 'tel_43', 
                  'tel_48', 'tel_49']

In [164]:
print(len(features_list))

26


In [138]:
train_X = train.loc[:, features_list]
train_y = train.iloc[:, -1]               

In [ ]:
test_X = test.loc[:, features_list]
test_X.head()

In [140]:
train_X = train_X.replace(to_replace='?',value=np.nan)
test_X = test_X.replace(to_replace='?',value=np.nan)
print(train_X.isnull().sum(axis=0).head(50))
print(test_X.isnull().sum(axis=0).head(50))

encounter_id                  0
patient_id                    0
race                        337
gender                        0
age                           0
admission_type_id             0
discharge_disposition_id      0
admission_source_id           0
time_in_hospital              0
tel_3                         0
tel_4                         0
tel_5                         0
tel_6                         0
tel_7                         0
tel_8                         0
tel_9                         1
tel_10                       33
tel_11                      139
tel_12                        0
tel_13                        0
tel_14                        0
tel_15                        0
tel_18                        0
tel_43                        0
tel_48                        0
tel_49                        0
dtype: int64
encounter_id                  0
patient_id                    0
race                        194
gender                        0
age                        

In [141]:
train_cat_cols=list(train_X.select_dtypes(include=['object']).columns)
train_num_cols=list(train_X.select_dtypes(exclude=['object']).columns)
test_cat_cols=list(test_X.select_dtypes(include=['object']).columns)
test_num_cols=list(test_X.select_dtypes(exclude=['object']).columns)

In [142]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(train_X)
train_X_df = imputer.transform(train_X)
test_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
test_imputer.fit(test_X)
test_X_df = imputer.transform(test_X)

In [143]:
#print(type(X_df))
train_X_df_pd=pd.DataFrame(train_X_df,columns=features_list)
test_X_df_pd=pd.DataFrame(test_X_df,columns=features_list)

In [144]:
train_X_df_pd.race.value_counts()

Caucasian          11553
AfricanAmerican     2543
Hispanic             278
Other                219
Asian                103
Name: race, dtype: int64

In [145]:
from sklearn.preprocessing import LabelEncoder
train_data1 = train_X_df_pd[train_cat_cols].apply(LabelEncoder().fit_transform)
test_data1 = test_X_df_pd[test_cat_cols].apply(LabelEncoder().fit_transform)

In [146]:
#dataf=data1.join(X_df_pd[num_cols])
train_dataf=(train_X_df_pd[train_num_cols]).join(train_data1)
test_dataf=(test_X_df_pd[test_num_cols]).join(test_data1)

In [ ]:
test_dataf.head()

In [ ]:
train_dataf.head()

In [149]:
train_X_final=train_dataf
test_X_final=test_dataf
train_y_final=train_y.values

In [150]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_X_final_std = sc.fit_transform(train_X_final)
test_X_final_std = sc.transform(test_X_final)

In [151]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(train_X_final, train_y_final)#y_train is default standardized with 0 and 1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [152]:
test_y_pred = classifier.predict(test_X_final_std)

In [153]:
features_list=['encounter_id', 'patient_id', 'race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id',
                  'admission_source_id', 'time_in_hospital', 'tel_3', 'tel_4', 'tel_5', 'tel_6', 'tel_7',
                  'tel_8', 'tel_9', 'tel_10', 'tel_11', 'tel_12', 'tel_13', 'tel_14', 'tel_15', 'tel_18', 'tel_43', 
                  'tel_48', 'tel_49']

In [154]:
features_list.remove('encounter_id')

In [156]:
test_X_final_only_eid=test_X_final.drop(features_list,axis=1)
test_X_final_only_eid.head()
test_y_pred_df=pd.DataFrame(test_y_pred)
test_y_pred_df.head()
final_df=pd.concat([test_X_final_only_eid,test_y_pred_df],axis=1)
final_df.columns=['encounter_id','diabetesMed']
print(final_df.head())
final_df.diabetesMed.value_counts()

     encounter_id  diabetesMed
7965     86749284            0
7966    135741342            0
7967     65660634            0
7968     73248582            0
7969     48791304            0


0    6501
1    1469
Name: diabetesMed, dtype: int64

In [168]:
print(final_df.dtypes)
convert_dict = {'encounter_id': object, 
                'diabetesMed': int
}
df = final_df.astype(convert_dict)

encounter_id     object
diabetesMed     float64
dtype: object


ValueError: ignored

In [157]:
train_X_final_only_eid=train_X_final.drop(features_list,axis=1)
train_X_final_only_eid.head()
test_y_pred_df=pd.DataFrame(test_y_pred)
test_y_pred_df.head()
final_df=pd.concat([train_X_final_only_eid,test_y_pred_df],axis=1)
final_df.columns=['encounter_id','diabetesMed']
print(final_df.head(15))
final_df.diabetesMed.value_counts()

   encounter_id  diabetesMed
0     178974720          1.0
1     111231666          0.0
2     134751624          0.0
3     254196636          1.0
4     432617750          0.0
5     139109832          0.0
6     290639712          1.0
7     266395878          0.0
8     140215032          0.0
9     124132824          0.0
10    360825710          0.0
11    130411104          0.0
12    131913252          0.0
13    158271282          1.0
14    125289972          0.0


0.0    6501
1.0    1469
Name: diabetesMed, dtype: int64

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.2, random_state = 1)

In [ ]:
print(type(X_train))
print(type(y_train))
print(type(X_test))
print(type(y_test))
#y.value_counts()
#type(y_test)
#type(X_final)
#pd.DataFrame(y_test, columns = features)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
print(X_train_std)

[[ 1.52880495  0.72127335  0.87071818 ... -0.09984252  1.13865811
   0.85112406]
 [-0.6579293  -1.08512553 -0.69648123 ... -0.09984252 -0.87822674
   0.85112406]
 [ 1.68932376  2.39711334 -0.69648123 ... -0.09984252 -0.87822674
   0.85112406]
 ...
 [-0.99077548 -0.92562194 -0.69648123 ... -0.09984252  1.13865811
   0.85112406]
 [-1.37814751  0.90301791  0.87071818 ... -0.09984252  1.13865811
   0.85112406]
 [-1.00140031  0.58802339  3.22151729 ... -0.09984252  1.13865811
  -0.61407062]]


In [ ]:
print(X_test_std)

[[ 1.53449826  1.32643265  0.87071818 ... -0.09984252 -0.87822674
  -2.0792653 ]
 [ 0.53490649 -0.74121888  0.87071818 ... -0.09984252 -0.87822674
   0.85112406]
 [-0.30073441 -0.92414849 -0.69648123 ... -0.09984252  1.13865811
   0.85112406]
 ...
 [-0.08028629 -0.69490992  0.87071818 ... -0.09984252 -0.87822674
   0.85112406]
 [-0.63340405 -0.51699842 -0.69648123 ... -0.09984252 -0.87822674
  -2.0792653 ]
 [ 0.21436906 -1.29183982 -0.69648123 ... -0.09984252 -0.87822674
  -0.61407062]]


In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train_std, y_train)#y_train is default standardized with 0 and 1

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = classifier.predict(X_test_std)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [1 1]
 [1 1]
 ...
 [1 1]
 [1 1]
 [1 1]]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[  88  218]
 [  77 2557]]


0.8996598639455783

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.53      0.29      0.37       306
           1       0.92      0.97      0.95      2634

    accuracy                           0.90      2940
   macro avg       0.73      0.63      0.66      2940
weighted avg       0.88      0.90      0.89      2940



In [ ]:
X_test_std_df = pd.DataFrame(X_test_std)
X_test_std_df.head()
#X_test_with_pred_df=pd.concat([X_test_std_df,y_test_df],axis=1)
#X_test_with_pred_df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,1.534498,1.326433,0.870718,0.636225,-1.270618,0.237876,-0.231452,0.981055,2.586063,-0.316756,0.797804,-0.513207,0.692530,0.370191,-0.928541,-0.750135,1.047630,-0.917749,1.181360,0.03445,0.20062,-0.509553,-0.027679,-0.099843,-0.878227,-2.079265
1,0.534906,-0.741219,0.870718,-0.489624,-1.270618,-1.135299,-1.170988,1.562956,-0.843033,-0.316756,-0.272124,-0.513207,-0.958308,0.370191,1.076958,-0.750135,-0.317010,-0.217990,2.575219,0.03445,0.20062,-0.509553,-0.027679,-0.099843,-0.878227,0.851124
2,-0.300734,-0.924148,-0.696481,-0.489624,0.501419,0.924463,0.312490,0.399154,-0.475630,-0.316756,-0.272124,-0.513207,0.692530,0.370191,1.076958,-0.750135,-0.420002,1.172557,-0.086500,0.03445,0.20062,1.656924,-0.027679,-0.099843,1.138658,0.851124
3,-0.474392,-0.699438,-0.696481,0.636225,0.501419,0.237876,-0.330350,-0.764649,-0.598098,-0.316756,-0.272124,1.062264,-2.609145,0.370191,-0.928541,1.172662,-0.059531,-0.209019,-1.173238,0.03445,0.20062,-0.509553,-0.027679,-0.099843,1.138658,-0.614071
4,-0.664210,-1.026528,-0.696481,-0.489624,0.501419,0.237876,0.065244,0.981055,-0.108227,0.375526,-0.272124,0.274528,0.692530,-2.038868,1.076958,0.531730,-0.059531,-0.289760,2.527970,0.03445,0.20062,-0.509553,-0.027679,-0.099843,1.138658,-0.614071


In [ ]:
type(X_test)
features_list.remove('encounter_id')

In [ ]:
X_test_only_eid = X_test.drop(features_list,axis=1)

In [ ]:
X_test_only_eid.head()
X_test.head()

,encounter_id,patient_id,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,tel_3,tel_4,tel_5,tel_6,tel_7,tel_8,tel_12,race,gender,age,tel_9,tel_10,tel_11,tel_13,tel_14,tel_15,tel_18,tel_43,tel_48,tel_49
10346,356646614,114025824,3,6,1,5,38,3,38,0,1,0,9,2,0,5,375,99,349,2,2,1,0,0,0,0
1927,269200656,41179185,3,1,1,1,19,4,10,0,0,0,6,2,1,5,216,177,526,2,2,1,0,0,0,2
1858,196097394,34734285,1,1,7,7,49,2,13,0,0,0,9,2,1,5,204,332,188,2,2,2,0,0,1,2
1429,180905508,42651189,1,6,7,5,36,0,12,0,0,2,3,2,0,8,246,178,50,2,2,1,0,0,1,1
8554,164299902,31127310,1,1,7,5,44,3,16,1,0,1,9,0,1,7,246,169,520,2,2,1,0,0,1,1


In [ ]:
print(features_list)

['patient_id', 'race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'time_in_hospital', 'tel_3', 'tel_4', 'tel_5', 'tel_6', 'tel_7', 'tel_8', 'tel_9', 'tel_10', 'tel_11', 'tel_12', 'tel_13', 'tel_14', 'tel_15', 'tel_18', 'tel_43', 'tel_48', 'tel_49']


In [ ]:
y_test_df=pd.DataFrame(y_test)
y_test_df
X_test_stadardized_df=pd.DataFrame(X_test)

In [ ]:
y_test_df.head()

,0
0,1
1,1
2,1
3,1
4,1


In [ ]:
pd.concat([X_test_only_eid,y_test_df],axis=1)

,encounter_id,0
0,NaN,1.0
1,NaN,1.0
2,NaN,1.0
3,NaN,1.0
4,432617750,1.0
...,...,...
14669,276215436,NaN
14670,166252908,NaN
14673,126709542,NaN
14684,106769232,NaN
